In [155]:
###---------LIBRARIES---------------------#####
from msilib.schema import Component
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, Input, Output   # pip install dash (version 2.0.0 or higher)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


## 0. Setting DataFrame & Filters

In [159]:
# DataFramee
df = pd.read_csv('data.csv')
df.drop_duplicates(keep = 'first', inplace = True)
df.usage_date = df.usage_date.apply(lambda x: dt.datetime.strptime(x.split()[0], '%Y-%m-%d'))
df.user_created_date = df.user_created_date.apply(lambda x: dt.datetime.strptime(x.split()[0], '%Y-%m-%d')if x is not np.nan else x)
df = df.set_index('usage_date',drop = True)[['cust_id', 'source_duration', 'transcoded_duration', 'video_duration',"user_created_date"]].sort_index(ascending = True)
df

,cust_id,source_duration,transcoded_duration,video_duration,user_created_date
usage_date,,,,,
2022-01-01,e356,99.058293,396.233171,NaN,2021-08-08
2022-01-01,475,1355.033724,5420.269792,NaN,2021-11-17
2022-01-01,4640,63.711967,255.114535,NaN,2021-11-18
2022-01-01,87a9,64.999300,259.997200,NaN,2021-12-13
2022-01-01,e36a,2210.154948,8840.619792,NaN,NaT
...,...,...,...,...,...
2022-10-21,31d2,1774.940495,5324.625000,NaN,NaT
2022-10-21,3fcf,28.658852,100.504688,NaN,2021-06-28
2022-10-21,5e4f,42659.866670,59163.225000,NaN,NaT


In [171]:
list(set(df.cust_id.to_list()))

['9777',
 'e83f',
 '4bcf',
 '6f88',
 '8e1d',
 '4805',
 'f2b2',
 '617d',
 'e040',
 '4ca0',
 '3ab6',
 'e2d5',
 '7132',
 '4cef',
 'e263',
 '1314',
 '7eb0',
 '0cb0',
 'e04e',
 'ebd5',
 'f400',
 '8d2c',
 '52d6',
 '99a1',
 'eccc',
 'b027',
 '3703',
 'c209',
 '7eb8',
 'df01',
 '3323',
 'd275',
 'f389',
 'bed7',
 'f65d',
 '675e',
 'f332',
 'e1e1',
 '2378',
 '64f6',
 'a602',
 '6a3b',
 'f192',
 '8136',
 '57ef',
 '01b6',
 '7.20E+04',
 '3.00E+67',
 '830f',
 'ee0e',
 '9867',
 '460a',
 'b9be',
 '5.90E+08',
 'ae33',
 '4d5c',
 'e832',
 'e7d7',
 '9907',
 'f111',
 '04d1',
 '060f',
 '24f2',
 '0b30',
 'efc8',
 '1cbd',
 'f987',
 'e4fd',
 '3-Feb',
 'd387',
 '6b66',
 'f681',
 'cddb',
 '47d4',
 'e301',
 '6a1c',
 '6ba5',
 '4f4f',
 'eeb2',
 '41bc',
 'f912',
 '2ef1',
 '5378',
 '3308',
 'b925',
 '9df3',
 '2e0c',
 '1484',
 '8d8d',
 'f050',
 '8bca',
 '975b',
 '637b',
 'fdf5',
 '7555',
 'b0f7',
 'ae4b',
 '2bf2',
 '10a4',
 '1e7d',
 '3.00E+53',
 '4758',
 '9ce3',
 '501d',
 'f3bc',
 'b1dd',
 '7.00E+97',
 '7819',
 '3f39'

In [157]:
# Filters

#Date Granularity - Dropdown List
date_granularity = {'Day': 'D', 'Month': 'M', 'Year': 'Y'}

#DateRange
start_date = dt.date(2022,1,1) ; end_date = dt.date(2022,10,21)

# Top N - Dropdown List
top =  5 #[5,10,20,30,40,50]

# Defining a filtered dataset
df_filter = df.loc[start_date:end_date].to_period(date_granularity['Day']).reset_index()
df_filter = df_filter.reset_index(drop = True)
df_filter.usage_date = df_filter.usage_date.apply(lambda x: x.to_timestamp())
data_dict = df_filter.to_dict(orient = 'record')
df = pd.DataFrame(data_dict)
df = df.set_index('usage_date')
df

,cust_id,source_duration,transcoded_duration,video_duration,user_created_date
usage_date,,,,,
2022-01-01,e356,99.058293,396.233171,NaN,2021-08-08
2022-01-01,475,1355.033724,5420.269792,NaN,2021-11-17
2022-01-01,4640,63.711967,255.114535,NaN,2021-11-18
2022-01-01,87a9,64.999300,259.997200,NaN,2021-12-13
2022-01-01,e36a,2210.154948,8840.619792,NaN,NaT
...,...,...,...,...,...
2022-10-21,31d2,1774.940495,5324.625000,NaN,NaT
2022-10-21,3fcf,28.658852,100.504688,NaN,2021-06-28
2022-10-21,5e4f,42659.866670,59163.225000,NaN,NaT


In [164]:
# Metrics Accumulated
df_total = df.sum(axis = 0, numeric_only = True)
df_total

source_duration        8.763605e+07
transcoded_duration    1.375423e+08
video_duration         2.250465e+06
dtype: float64

In [158]:
df_potencial = df.copy()
df_potencial['new_customer'] = df_potencial.user_created_date.apply(lambda x:
                                                                    True if x > df_potencial.index[-1] - dt.timedelta(days = 90)
                                                                  else False)
df_potencial

,cust_id,source_duration,transcoded_duration,video_duration,user_created_date,new_customer
usage_date,,,,,,
2022-01-01,e356,99.058293,396.233171,NaN,2021-08-08,False
2022-01-01,475,1355.033724,5420.269792,NaN,2021-11-17,False
2022-01-01,4640,63.711967,255.114535,NaN,2021-11-18,False
2022-01-01,87a9,64.999300,259.997200,NaN,2021-12-13,False
2022-01-01,e36a,2210.154948,8840.619792,NaN,NaT,False
...,...,...,...,...,...,...
2022-10-21,31d2,1774.940495,5324.625000,NaN,NaT,False
2022-10-21,3fcf,28.658852,100.504688,NaN,2021-06-28,False
2022-10-21,5e4f,42659.866670,59163.225000,NaN,NaT,False


In [136]:
df_filter = df_filter.reset_index()
df_filter.usage_date = df_filter.usage_date.apply(lambda x: x.to_timestamp())
data_dict = df_filter.to_dict(orient = 'record')
df = pd.DataFrame(data_dict)
df = df.set_index('usage_date')

In [139]:
df_potencial = df.copy()
df_potencial['new_customer'] = df_potencial.user_created_date.apply(lambda x:
                                                                    True if x > df_potencial.index[-1]- dt.timedelta(days = 90)
                                                                   else False)
df_new = df_potencial[df_potencial.new_customer == True]

In [ ]:
df

## 1. Metrics Over Time

In [5]:
df_overtime = df_filter.groupby(by = 'usage_date').agg({'source_duration': np.sum,
                                                        'transcoded_duration': np.sum,
                                                        'video_duration': np.sum})
df_overtime

,source_duration,transcoded_duration,video_duration
usage_date,,,
2022-01-01,277516.948135,425663.449481,0.000000
2022-01-02,277795.558304,394583.587921,0.000000
2022-01-03,260282.767226,364014.982308,0.000000
2022-01-04,267185.777589,378787.381206,0.000000
2022-01-05,305028.585976,448857.249877,0.000000
...,...,...,...
2022-10-17,275351.046731,437041.385764,140.959395
2022-10-18,277086.465503,439435.580519,205.737614
2022-10-19,262383.156130,436053.416383,439.125033


## 2. Metrics -   Total Running

In [6]:
df_cumsum = df_filter.groupby(by = 'usage_date').agg({'source_duration': np.sum,
                                                      'transcoded_duration': np.sum,
                                                      'video_duration': np.sum}).cumsum(axis =0)
df_cumsum 

,source_duration,transcoded_duration,video_duration
usage_date,,,
2022-01-01,2.775169e+05,4.256634e+05,0.000000e+00
2022-01-02,5.553125e+05,8.202470e+05,0.000000e+00
2022-01-03,8.155953e+05,1.184262e+06,0.000000e+00
2022-01-04,1.082781e+06,1.563049e+06,0.000000e+00
2022-01-05,1.387810e+06,2.011907e+06,0.000000e+00
...,...,...,...
2022-10-17,8.677648e+07,1.361517e+08,2.249471e+06
2022-10-18,8.705357e+07,1.365912e+08,2.249677e+06
2022-10-19,8.731595e+07,1.370272e+08,2.250116e+06


## 3. Metrics - Accumulated

In [7]:
df_total = df_filter.sum(axis = 0, numeric_only = True)
df_total

source_duration        8.763605e+07
transcoded_duration    1.375423e+08
video_duration         2.250465e+06
dtype: float64

## 4. Metrics - Variation Over Time

In [8]:
df_pct_change = df_filter.groupby(by = 'usage_date').agg({'source_duration': np.sum,
                                    'transcoded_duration': np.sum,
                                    'video_duration': np.sum}).pct_change().drop(df_filter.index[0],axis = 0).replace(np.inf,0)
df_pct_change 

,source_duration,transcoded_duration,video_duration
usage_date,,,
2022-01-02,0.001004,-0.073015,NaN
2022-01-03,-0.063042,-0.077471,NaN
2022-01-04,0.026521,0.040582,NaN
2022-01-05,0.141635,0.184985,NaN
2022-01-06,-0.029673,-0.065747,NaN
...,...,...,...
2022-10-17,-0.039343,-0.014368,0.666746
2022-10-18,0.006303,0.005478,0.459552
2022-10-19,-0.053064,-0.007697,1.134394


## 5. Metrics - Ranking

In [16]:
#top = 10
column_name = 'transcoded_duration'
df_rank = df_filter.groupby(by = 'cust_id').agg({'source_duration': np.sum,
                                    'transcoded_duration': np.sum,
                                    'video_duration': np.sum})[[column_name ]].sort_values(by =column_name,
                                                                                                   ascending = False)
df_top = df_rank.copy()
df_top['ranking'] = range(1,len(df_top.index) +1)
df_top = df_top.iloc[:top,:].reset_index().assign(rank_id = lambda x: x['ranking'].apply(lambda x: str(x) + '-) ') + x['cust_id'])
df_top

,cust_id,transcoded_duration,ranking,rank_id
0,5e4f,1.085833e+08,1,1-) 5e4f
1,8863,5.038736e+06,2,2-) 8863
2,3fcf,4.428012e+06,3,3-) 3fcf
3,e36a,3.215696e+06,4,4-) e36a
4,31d2,3.103342e+06,5,5-) 31d2


,cust_id,transcoded_duration,ranking,rank_id
0,5e4f,1.085833e+08,1,1-) 5e4f
1,8863,5.038736e+06,2,2-) 8863
2,3fcf,4.428012e+06,3,3-) 3fcf
3,e36a,3.215696e+06,4,4-) e36a
4,31d2,3.103342e+06,5,5-) 31d2


In [137]:
df_rank

,transcoded_duration
cust_id,
5e4f,1.085833e+08
8863,5.038736e+06
3fcf,4.428012e+06
e36a,3.215696e+06
31d2,3.103342e+06
...,...
b301,0.000000e+00
b291,0.000000e+00
b1dd,0.000000e+00
